In [1]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().resolve()
if ROOT.name == "Note": ROOT = ROOT.parents[0]
CHAN = ROOT / "chan_data"

def load_trading(sym: str) -> pd.DataFrame:
    p = CHAN/"signals"/"trading"/sym/"multi_tf_trading_signals.csv"
    df = pd.read_csv(p, parse_dates=["ts"])
    for c in ("tf","side","tag","notes","nested_state","upper_tf"):
        if c in df.columns: df[c] = df[c].astype(str)
    return df.sort_values("ts").reset_index(drop=True)

def summary_trading(sym: str, top_n: int = 10, thr=1.6):
    df = load_trading(sym)
    print(f"[{sym}] total={len(df)}  score_final>= {thr}: {(df['score_final']>=thr).sum()}")
    print(df.groupby(["tf","side","tag"]).size().sort_values(ascending=False).head(12))
    print("\n-- Top by score_final --")
    print(df.sort_values("score_final", ascending=False)
            .head(top_n)[["ts","tf","side","tag","price","score_final","resonance_n","nested_state","notes"]])

summary_trading("BTCUSDT", top_n=8, thr=1.6)
summary_trading("ETHUSDT", top_n=8, thr=1.6)

[BTCUSDT] total=259  score_final>= 1.6: 65
tf   side  tag
15m  buy   3B     20
30m  buy   3B     18
     sell  1S     17
5m   buy   3B     16
15m  sell  3S     15
5m   buy   1B     15
1h   buy   3B     15
30m  sell  3S     14
5m   sell  1S     13
15m  buy   1B     13
     sell  1S     13
5m   sell  3S     12
dtype: int64

-- Top by score_final --
                           ts   tf  side tag      price  score_final  \
27  2025-04-13 01:30:00+00:00  15m  sell  1S   85726.99     2.879671   
26  2025-04-13 01:00:00+00:00   1h  sell  1S   85259.77     2.717695   
244 2025-10-05 05:30:00+00:00  30m  sell  1S  125127.99     2.370305   
6   2025-04-02 21:00:00+00:00   1h  sell  1S   84855.99     2.331200   
46  2025-05-06 13:30:00+00:00  30m   buy  1B   93616.92     2.325200   
74  2025-05-19 14:30:00+00:00  30m   buy  1B  104328.11     2.300927   
167 2025-08-01 08:00:00+00:00   1h   buy  1B  114615.99     2.297945   
171 2025-08-02 19:00:00+00:00  15m   buy  1B  112046.90     2.280000   

  

In [3]:
import pandas as pd
from pathlib import Path

ROOT = Path.cwd().resolve()
if ROOT.name == "Note": ROOT = ROOT.parents[0]
DATA = ROOT / "DATA"
CHAN = ROOT / "chan_data"

def read_k(sym, tf):
    df = pd.read_csv(DATA/sym/f"{tf}.csv")
    if "timestamp" not in df.columns:
        for c in ("time","datetime","ts"):
            if c in df.columns: df = df.rename(columns={c:"timestamp"}); break
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
    return df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)

def inspect_signal(sym: str, when: str, tf: str, left=50, right=30):

    df = read_k(sym, tf)
    ts = pd.to_datetime(when, utc=True)
    idx = pd.to_datetime(df["timestamp"]).searchsorted(ts)
    i0 = max(0, idx-left); i1 = min(len(df)-1, idx+right)
    win = df.iloc[i0:i1+1].copy()
    mark = win[win["timestamp"]==ts]
    print(f"[{sym}-{tf}] window {i0}:{i1}, ts in window={not mark.empty}")
    cols = [c for c in ("timestamp","open","high","low","close","volume") if c in win.columns]
    return win[cols]

w = inspect_signal("BTCUSDT", "2025-10-05 05:30:00+00:00", "15m")
w.head(10)


[BTCUSDT-15m] window 18519:18599, ts in window=True


timestamp       open       high        low      close  \
18519 2025-10-04 17:00:00+00:00  121720.00  121846.00  121660.57  121779.83   
18520 2025-10-04 17:15:00+00:00  121779.82  121862.34  121734.69  121846.53   
18521 2025-10-04 17:30:00+00:00  121846.53  121910.24  121678.33  121718.90   
18522 2025-10-04 17:45:00+00:00  121718.90  121747.37  121669.22  121747.37   
18523 2025-10-04 18:00:00+00:00  121747.36  121829.69  121700.01  121704.13   
18524 2025-10-04 18:15:00+00:00  121704.13  121760.48  121700.19  121732.10   
18525 2025-10-04 18:30:00+00:00  121732.11  121732.11  121596.39  121677.82   
18526 2025-10-04 18:45:00+00:00  121677.82  121677.83  121510.00  121574.66   
18527 2025-10-04 19:00:00+00:00  121574.66  121800.98  121574.65  121800.98   
18528 2025-10-04 19:15:00+00:00  121800.98  121866.87  121754.01  121826.01   

          volume  
18519   83.75210  
18520   42.75780  
18521  117.63152  
18522   45.19321  
18523   79.05410  
18524   54.59960  
18525   65.03849  
18526   89.42900  
18527   82.96017  
18528   34.15518

In [4]:
from datetime import timedelta

def export_alerts(sym: str, min_score=1.8, lookback_hours=48, outfile="alerts.csv"):
    df = load_trading(sym)
    tmax = pd.to_datetime(df["ts"]).max()
    t0 = tmax - pd.Timedelta(hours=lookback_hours)
    pick = df[(pd.to_datetime(df["ts"])>=t0) & (df["score_final"]>=min_score)].copy()
    pick = pick.sort_values(["ts","score_final"], ascending=[True, False])
    outdir = CHAN/"signals"/"alerts"/sym
    outdir.mkdir(parents=True, exist_ok=True)
    p = outdir/outfile
    pick.to_csv(p, index=False)
    print(f"[ALERTS] {sym}: {len(pick)} -> {p}")
    return pick

_ = export_alerts("BTCUSDT", min_score=1.8, lookback_hours=72)
_ = export_alerts("ETHUSDT", min_score=1.8, lookback_hours=72)

[ALERTS] BTCUSDT: 1 -> /Users/cc/PycharmProjects/ChanLab/chan_data/signals/alerts/BTCUSDT/alerts.csv
[ALERTS] ETHUSDT: 1 -> /Users/cc/PycharmProjects/ChanLab/chan_data/signals/alerts/ETHUSDT/alerts.csv
